In [ ]:
#default_exp export

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.data.all import *
from fastai.learner import *

from fastexport.jit import *

In [ ]:
#export
@patch
def to_fastai(self:Learner, fname='export', pickle_module=pickle, pickle_protocol=2, **kwargs):
    torch.save(self, self.path/fname+'.pkl', pickle_module=pickle_module, pickle_protocol=pickle_protocol)

In [ ]:
#export
@patch
def export(self:Learner, fname='export', mode='fastai', with_dls=True, pickle_module=pickle, pickle_protocol=2):
    "Export a `Learner` with `mode` and potentially `dls`"
    if rank_distrib(): return # don't export if child proc
    self._end_cleanup()
    old_dls = self.dls
    self.dls = self.dls.new_empty()
    state = self.opt.state_dict() if self.opt is not None else None
    self.opt = None
    with warnings.catch_warnings():
        #To avoid the warning that come from PyTorch about model not being checked
        warnings.simplefilter("ignore")
        getattr(self, f'to_{mode}')(fname, mode=mode)
        if with_dls and mode != 'fastai': # handle ts, onnx
            torch.save(self.dls, self.path/fname+'.pkl', pickle_module=pickle_module, pickle_protocol=pickle_protocol)